In [ ]:
import dt_ids7_export_module as bh_utils
import mapping_module as bh_map
import plot_module as bh_plot

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'C:\\' # 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = ''


# Import the IDS7 data:
root_folder = path + "/run/media/bhosteras/PRIVATE_USB/Intervensjon Doser/IDS7/2024/"
df_ids7 = bh_utils.import_excel_files_to_dataframe(root_folder)

# Import the IDS7 data:
root_folder = path + "/run/media/bhosteras/PRIVATE_USB/Intervensjon Doser/DoseTrack - Serienivå/2024/"
df_dt = bh_utils.import_excel_files_to_dataframe(root_folder)

In [ ]:
# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Merge the dataframes:
data = bh_utils.merge_ids7_dt(df_ids7, df_dt, True)


In [ ]:
# Get the mapping dictionary:
import mapping_dict_DSA as dsa_dict
import importlib
importlib.reload(dsa_dict)
mapping_dict = dsa_dict.get_DSA_mapping_dict()
data = bh_map.map_procedures(data, mapping_dict, True)

In [ ]:
import pandas as pd
from IPython.display import display

# Ensure Age is numeric
data['Age (Years)'] = pd.to_numeric(data['Age (Years)'], errors='coerce')

# Define aggregation logic
def q1(x): return x.quantile(0.25)
def q3(x): return x.quantile(0.75)

agg_spec = {
    'Count': 'count',
    'Median': 'median',
    'Mean': 'mean',
    'Q1': q1,
    'Q3': q3
}

# Define columns to round
cols_to_round = ['Median', 'Mean', 'Q1', 'Q3']

# 1. Stratify by Age < 16
df_under_16 = data[data['Age (Years)'] < 16]
print(f"Number of procedures for Age < 16: {len(df_under_16)}")

if len(df_under_16) > 0:
    stats_under_16 = df_under_16.groupby('Mapped Procedures')['DAP Total (Gy*cm2)'].agg(**agg_spec)
    # Round specific columns
    stats_under_16[cols_to_round] = stats_under_16[cols_to_round].round(3)
    # Sort alphabetically by index (Mapped Procedures)
    stats_under_16 = stats_under_16.sort_index()
    print("\n--- Statistics for Age < 16 (DAP in Gy*cm2) ---")
    display(stats_under_16)
else:
    print("No data for Age < 16")

# 2. Stratify by Age >= 16
df_over_16 = data[data['Age (Years)'] >= 16]
print(f"\nNumber of procedures for Age >= 16: {len(df_over_16)}")

if len(df_over_16) > 0:
    stats_over_16 = df_over_16.groupby('Mapped Procedures')['DAP Total (Gy*cm2)'].agg(**agg_spec)
    # Round specific columns
    stats_over_16[cols_to_round] = stats_over_16[cols_to_round].round(3)
    # Sort alphabetically by index (Mapped Procedures)
    stats_over_16 = stats_over_16.sort_index()
    print("\n--- Statistics for Age >= 16 (DAP in Gy*cm2) ---")
    display(stats_over_16)


In [ ]:
median_int03 = data[(data['Mapped Procedures'] == 'INT03 - Hjerte') & (data['Age (Years)'] >= 16)]['DAP Total (Gy*cm2)'].median()
print(f"Median DAP for INT03 (Age >= 16): {median_int03}")